In [4]:
!pip3 install gymnasium
!pip3 install numpy
!pip3 install IPython
!pip3 install matplotlib
!pip3 install keras


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [5]:
!pip3 install tensorflow

# **Applying Needed Libraries**

In [6]:
import gymnasium as gym
import numpy as np
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from IPython import display

# **Arranging Params**

In [7]:

# Deep Q-Learning parametreleri
gamma = 0.95  # İndirim oranı
epsilon = 1.0  # Keşif oranı başlangıç değeri
epsilon_min = 0.01  # Minimum keşif oranı
epsilon_decay = 0.995  # Keşif oranı azalma hızı
learning_rate = 0.001  # Öğrenme hızı
batch_size = 64  # Mini-batch boyutu
memory_size = 2000  # Replay buffer boyutu

# **Creating Environment & Neural Links**

In [8]:
# Ortamı oluştur
import gymnasium as gym

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]  # Durum uzayının boyutu
action_size = env.action_space.n  # Aksiyon uzayının boyutu

# Replay buffer (deneyim tekrarı için)
memory = deque(maxlen=memory_size)  # FIFO veri yapısı

# Q-ağını (sinir ağı) oluştur
model = Sequential([
    Input(shape=(state_size,)),
    Dense(24, activation='relu'),  # İlk gizli katman
    Dense(24, activation='relu'),  # İkinci gizli katman
    Dense(action_size, activation='linear')  # Çıkış katmanı
])

model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))  # Modeli derle


# **Policy & Replay Buffer**

In [9]:
# Ajanın hareketini belirleyen politika fonksiyonu
def act(state):
    if np.random.rand() <= epsilon:  # Epsilon-greedy politika
        return env.action_space.sample()  # Rastgele aksiyon seç
    act_values = model.predict(state)  # Modelden aksiyon değerlerini tahmin et
    return np.argmax(act_values[0])  # En iyi aksiyonu seç

# Deneyimleri replay buffer'a ekleme ve ağın eğitimi
def replay():
    global epsilon
    if len(memory) < batch_size:  # Replay buffer yeterince büyük değilse geri dön
        return
    minibatch = random.sample(memory, batch_size)  # Replay buffer'dan rastgele örnekler al
    for state, action, reward, next_state, done in minibatch:
        target = reward  # Hedef ödül
        if not done:
            target = (reward + gamma * np.amax(model.predict(next_state)[0]))  # Hedefi güncelle
        target_f = model.predict(state)  # Mevcut tahmin
        target_f[0][action] = target  # Tahmini hedef ile güncelle
        model.fit(state, target_f, epochs=1, verbose=0)  # Modeli eğit
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay  # Epsilon'u azalt

# **Initializing Agent**

In [ ]:

# Ajanı çalıştır
episodes = 1000  # Eğitim döngüsü sayısı
scores = []  # Bölüm sonuçlarını depolamak için liste
for e in range(episodes):
    state = env.reset()  # Ortamı sıfırla ve başlangıç durumunu al  # ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
    state = np.reshape(state, [1, state_size])  # Durumu şekillendir
    score = 0  # Bölüm puanı
    for time in range(500):  # Her bölümde maksimum adım sayısı
        action = env.action_space.sample()  # Rastgele aksiyon seç
        next_state, reward, done, _ = env.step(action)  # Aksiyonu uygula ve yeni durumu al
        reward = reward if not done else -10  # Ödülü güncelle
        next_state = np.reshape(next_state, [1, state_size])  # Yeni durumu şekillendir
        state = next_state  # Durumu güncelle
        score += reward  # Puanı güncelle
        if done:  # Eğer bölüm sonlandıysa
            print(f"episode: {e}/{episodes}, score: {score}")  # Bölüm bilgisini yazdır
            scores.append(score)  # Bölüm sonucunu kaydet
            break

    # Her 50 bölümde bir performansı göster
    if e % 50 == 0:
        render(env)  # Ortamı görselleştir

env.close()  # Ortamı kapat

# **Visualisation**

In [ ]:
# Eğitim sonuçlarını çizdir
plt.figure(figsize=(10, 6))
plt.plot(scores, marker='o', linestyle='-', color='b')
plt.xlabel('Episode')
plt.ylabel('Score')
plt.title('Deep Q-Learning - CartPole-v1')
plt.grid(True)
plt.show()